# 1.导入包及数据

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
import pandas as pd
from sklearn import preprocessing
from vmdpy import VMD  
import akshare as ak #大豆数据集
from scipy.fftpack import fft ##傅里叶级数
import pkuseg   #分词包
import re
from snownlp import SnowNLP #中文情感分析包

from sklearn.feature_extraction.text import CountVectorizer #sklearn的统计词特征包
from sklearn.feature_extraction.text import TfidfTransformer #sklearn的TF-IDF包





#下载数据
Data=pd.read_csv("./data.csv")
Data.head()


In [ ]:
#数据处理
pd.to_datetime(Data.date)
Data.drop(columns={'Unnamed: 0'},inplace=True)
Data.head()

#### 标准化数据

In [ ]:
minmax = preprocessing.MinMaxScaler()
data_minmax = minmax.fit_transform(Data[['return','volatility']])
data_minmax_list = data_minmax.tolist()
data_minmax_list

In [ ]:
#建空列表把循环出来每次嵌套列表的数装在新的列表中
empt_return=[]
empt_volatility=[]
for i in range(len(data_minmax_list)):
    empt_return.append(data_minmax_list[i][0])
    empt_volatility.append(data_minmax_list[i][1])
Data['return']=empt_return
Data['volatility']=empt_volatility
Data 

# 2.VMD分解 Return&Volatility 

### vmd参数设置及绘制分解模态图

In [ ]:
#1.导入文件，可以指定列也可以，就是usecols
filename= 'data.csv'
f = pd.read_csv(filename,usecols=[9,10])

#vmd包参数设置并执行VMD
alpha = 5000       #宽带限制，一般为数据点的1.5-2倍  
tau = 0.           # 噪声限制 (no strict fidelity enforcement)  
K = 11             # 分解模态个数  
DC = 0             # 合成信号若无常量则为0；若含常量，则其取值为 1；  
init = 1           # 初始化ω值,当初始化为1时,均匀分布产生的随机数； 
tol = 1e-7         # 控制误差大小常量，决定精度与迭代次数

u, u_hat, omega = VMD(f['return'], alpha, tau, K, DC, init, tol)  
u1,u1_hat,omega1 = VMD(f['volatility'], alpha, tau, K, DC, init, tol)  
#绘制分解模态图
plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.plot(u.T)
plt.title('Decomposed modes of return')
plt.subplot(2,1,2)
plt.plot(u1.T)
plt.title('Decomposed modes of volatility')


### IMF of return

In [ ]:
#未分解前的图
fig_return = plt.figure()
plt.plot(f['return'])
fig_return.suptitle('Original input signal and its components')

#循环每个分解模态的
for i in range(K):
    plt.figure(figsize=(5,5), dpi=100)
    plt.subplot(K,1,i+1)
    plt.plot(u[i,:], linewidth=0.2, c='r')
    plt.ylabel('IMF{}'.format(i+1))
    
    #每个模态的中心频率
for i in range(K):
    plt.figure(figsize=(5,5), dpi=100)
    plt.subplot(K,1,i+1)
    plt.plot(abs(fft(u[i,:])))
    plt.ylabel('IMF{}'.format(i+1))



### IMF of volatility

In [ ]:
#未分解前的图
fig_volatility = plt.figure()
plt.plot(f['volatility'])
fig_volatility.suptitle('Original input signal and its components OF volatility')

#循环每个分解模态的
for i in range(K):
    plt.figure(figsize=(5,5), dpi=100)
    plt.subplot(K,1,i+1)
    plt.plot(u1[i,:], linewidth=0.2, c='r')
    plt.ylabel('IMF{}'.format(i+1))
    
    #每个模态的中心频率
for i in range(K):
    plt.figure(figsize=(5,5), dpi=100)
    plt.subplot(K,1,i+1)
    plt.plot(abs(fft(u1[i,:])))
    plt.ylabel('IMF{}'.format(i+1))

# 3.文本处理

In [ ]:
filepath1='./text.csv'
text=pd.read_csv(filepath1)
pd.to_datetime(text.date)

### (1).文本预处理

In [ ]:
titles=text.title
titles1=[]

for i in range(len(titles)):
    textgo = re.sub('[^\u4e00-\u9fa5_a-zA-Z0-9]','',str(titles[i]))
    titles1.append(textgo) 


In [ ]:
len(titles1)

###  (2).文本分词

In [ ]:
type(titles1)

In [ ]:
seg = pkuseg.pkuseg() # 以默认配置加载模型

In [ ]:
textcut=[]
for j in titles1:  
    textfo=seg.cut( j )  # 进行分词
    textcut.append(textfo)
    
text_cut=pd.Series(textcut)##list换series，然后放入表中

In [ ]:
text.insert(loc=1,column='text_cut',value=text_cut)

In [ ]:
text

###  (3).去除停用词（采用中文停用词表）

In [ ]:
def stopwordslist(filepath):   # 定义函数创建停用词列表
    stopword = [line.strip() for line in open(filepath,encoding='utf-8').readlines()]    #以行的形式读取停用词表，同时转换为列表
    return stopword

filepath='./cn_stopwords.txt'
stopwords = stopwordslist(filepath)

In [ ]:
text_cut[0][1]

In [ ]:
type(text_cut)

In [ ]:
lastsentences=[]
lastsentences_str=[]##增加列表内是字符串的一项，方便进行向量化处理
for i in range(len(text_cut)):     #for循环遍历分词后的每个词语
    lastsentence=[]
    for word in text_cut[i]:
        if word not in stopwords:     #判断分词后的词语是否在停用词表内
            lastsentence.append(word)
            lastsentence_str=' '.join(lastsentence)
    lastsentences.append(lastsentence)
    lastsentences_str.append(lastsentence_str)
text_cut_stopwords=pd.Series(lastsentences)
text_cut_stopwords_str=pd.Series(lastsentences_str)
text.insert(loc=2,column='text_cut_stopwords',value=text_cut_stopwords)
text.insert(loc=2,column='text_cut_stopwords_str',value=text_cut_stopwords_str)

In [ ]:
lastsentences_str

In [ ]:
text.text_cut_stopwords[0]

In [ ]:
text

###  (4).向量转换（BoW & TF-IDF）

In [ ]:
#CountVectorizer和TfidfTransformer默认学习list里的str（在str中以空格为分断），所以对象应该换成list(‘str1’,‘str2’,...)
vectorizer = CountVectorizer(min_df=1,max_df=0.8) ##创建单词表时，忽略占比超过百分之五十的词 以及 出现数少于2的词
transformer = TfidfTransformer() 
X = vectorizer.fit_transform(lastsentences_str)#学习词汇，返回文档术语与矩阵
feature_name = vectorizer.get_feature_names()#显示特征名
tfidf = transformer.fit_transform(X)  #把已经生成的文档矩阵换成TF-IDF矩阵
tfidf.toarray()

### (5).情感分析

In [ ]:
sentiment=[]
for i in range(len(text_cut_stopwords_str)):
    sentiment.append(SnowNLP(text_cut_stopwords_str[i]).sentiments)
text.insert(loc=5,column='sentiment',value=sentiment)
text

In [ ]:
text['date']=pd.to_datetime(text['date'])




In [ ]:
text['date']

In [ ]:
type(senti_all_date.index)

In [ ]:
type(text['date'])

In [ ]:
#创建对应日期数组
senti_date=pd.date_range('20100104','20190701')

In [ ]:
senti_all_date=pd.DataFrame(index=senti_date)

In [ ]:
senti_all_date





In [ ]:
#


In [ ]:
senti_all_date

In [ ]:
dic=


In [ ]:

dict(zip(text.date,text.sentiment))


In [139]:
text.date.value_counts()

2011-04-21    30
2012-05-16    30
2013-06-26    29
2012-05-10    28
2012-08-17    27
              ..
2011-06-22     1
2013-06-01     1
2011-01-04     1
2011-07-05     1
2012-05-03     1
Name: date, Length: 1183, dtype: int64

In [140]:
dict(zip(text.date,text.sentiment))


{Timestamp('2019-07-01 00:00:00'): 0.9883797770464233,
 Timestamp('2019-06-06 00:00:00'): 0.889135768466459,
 Timestamp('2019-05-13 00:00:00'): 0.9493189389916733,
 Timestamp('2019-04-25 00:00:00'): 0.4224865760847407,
 Timestamp('2019-04-08 00:00:00'): 0.8927261485639549,
 Timestamp('2019-02-12 00:00:00'): 0.9736327338488427,
 Timestamp('2019-01-18 00:00:00'): 0.8982244603344833,
 Timestamp('2019-01-03 00:00:00'): 0.9716483021451002,
 Timestamp('2018-12-28 00:00:00'): 0.9824485204727833,
 Timestamp('2018-12-25 00:00:00'): 0.9606021916735854,
 Timestamp('2018-11-26 00:00:00'): 0.4575371139919314,
 Timestamp('2018-11-09 00:00:00'): 0.0004323854623757839,
 Timestamp('2018-10-18 00:00:00'): 0.9383730874032127,
 Timestamp('2018-09-25 00:00:00'): 0.8437689387190664,
 Timestamp('2018-09-19 00:00:00'): 0.8437689387190664,
 Timestamp('2018-09-18 00:00:00'): 0.6675455468393046,
 Timestamp('2018-09-17 00:00:00'): 0.8966494047583785,
 Timestamp('2018-09-07 00:00:00'): 0.9953579535471005,
 Timesta

In [141]:
#查看20180806数据
text[text.date=='2018-08-06']

,title,text_cut,text_cut_stopwords_str,text_cut_stopwords,date,sentiment
19,国务院：决定对美600亿美元进口商品加征关税,"[国务院, 决定, 对, 美, 600亿, 美元, 进口, 商品, 加征, 关税]",国务院 决定 美 600亿 美元 进口 商品 加征 关税,"[国务院, 决定, 美, 600亿, 美元, 进口, 商品, 加征, 关税]",2018-08-06,0.979442
20,外交部长王毅：中美在大方向上达成一致,"[外交, 部长, 王毅, 中, 美, 在, 大方向上, 达成, 一致]",外交 部长 王毅 中 美 大方向上 达成 一致,"[外交, 部长, 王毅, 中, 美, 大方向上, 达成, 一致]",2018-08-06,0.985522


In [ ]:
senti_all_date['sentiment2']=senti_all_date.index.map(dict(zip(text.date,text.sentiment)),0.5)

In [130]:
senti_all_date['sentiment2'].isnull().sum()

2283

In [ ]:
type(text.date)


In [132]:
senti_all_date



,sentiment,sentiment2
2010-01-04,0,0.530890
2010-01-05,0,0.003893
2010-01-06,0,0.765290
2010-01-07,0,0.368800
2010-01-08,0,0.350436
...,...,...
2019-06-27,0,NaN
2019-06-28,0,NaN
2019-06-29,0,NaN
2019-06-30,0,NaN


In [133]:
text

,title,text_cut,text_cut_stopwords_str,text_cut_stopwords,date,sentiment
0,中美元首会晤，习近平同特朗普谈了这些大事,"[中, 美元, 首, 会晤, 习近平, 同, 特朗普, 谈, 了, 这些, 大事]",中 美元 首 会晤 习近平 特朗普 谈 大事,"[中, 美元, 首, 会晤, 习近平, 特朗普, 谈, 大事]",2019-07-01,0.988380
1,投机基金仍然看空大豆市场，但是过去一周减持净空单,"[投机, 基金, 仍然, 看空, 大豆, 市场, 但是, 过去, 一, 周, 减持, 净空单]",投机 基金 仍然 看空 大豆 市场 过去 周 减持 净空单,"[投机, 基金, 仍然, 看空, 大豆, 市场, 过去, 周, 减持, 净空单]",2019-06-06,0.889136
2,香港查出非洲猪瘟病毒感染政府销毁上水屠宰场600头生猪,"[香港, 查出, 非洲, 猪瘟, 病毒, 感染, 政府, 销毁, 上水, 屠宰场, 600,...",香港 非洲 猪瘟 病毒 感染 政府 销毁 上水 屠宰场 600 头 生猪,"[香港, 非洲, 猪瘟, 病毒, 感染, 政府, 销毁, 上水, 屠宰场, 600, 头, 生猪]",2019-05-13,0.949319
3,投机基金在CBOT大豆期市的净空单创下15个月新高,"[投机, 基金, 在, CBOT, 大豆, 期市, 的, 净空单, 创下, 15, 个, 月...",投机 基金 CBOT 大豆 期市 净空单 15 月 新高,"[投机, 基金, CBOT, 大豆, 期市, 净空单, 15, 月, 新高]",2019-04-25,0.422487
4,西藏自治区林芝市发生非洲猪瘟疫情,"[西藏, 自治区, 林芝市, 发生, 非洲, 猪瘟, 疫情]",西藏 自治区 林芝市 非洲 猪瘟 疫情,"[西藏, 自治区, 林芝市, 非洲, 猪瘟, 疫情]",2019-04-08,0.892726
...,...,...,...,...,...,...
3286,美国大豆进口订货量过高，或造成供过于求,"[美国, 大豆, 进口, 订货量, 过, 高, 或, 造成, 供过于求]",美国 大豆 进口 订货量 高 造成 供过于求,"[美国, 大豆, 进口, 订货量, 高, 造成, 供过于求]",2010-01-06,0.765290
3287,政策预期助推农产品期货走强,"[政策, 预期, 助推, 农产品, 期货, 走强]",政策 预期 助推 农产品 期货 走强,"[政策, 预期, 助推, 农产品, 期货, 走强]",2010-01-05,0.896063
3288,78家大豆加工企业暂被取消补贴资格,"[78, 家, 大豆, 加工, 企业, 暂, 被, 取消, 补贴, 资格]",78 家 大豆 加工 企业 暂 取消 补贴 资格,"[78, 家, 大豆, 加工, 企业, 暂, 取消, 补贴, 资格]",2010-01-05,0.003893
3289,关于东北大豆市场补贴消息播报,"[关于, 东北, 大豆, 市场, 补贴, 消息, 播报]",东北 大豆 市场 补贴 消息 播报,"[东北, 大豆, 市场, 补贴, 消息, 播报]",2010-01-04,0.086830
